In [ ]:
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline
from ovejero import forward_modeling, model_trainer
import os
import corner

# Modifies the paths in the config to agree with the paths being used on the current computer.
def recursive_str_checker(cfg_dict):
    for key in cfg_dict:
        if isinstance(cfg_dict[key],str):
            cfg_dict[key] = cfg_dict[key].replace('/home/swagnercarena/ovejero/',root_path)
        if isinstance(cfg_dict[key],dict):
            recursive_str_checker(cfg_dict[key])

# Comparing Performance to Forward Modeling

__Author:__ Sebastian Wagner-Carena

__Last Run:__ 07/27/2020

__Goals:__ Compare the performance of the BNN model to a forward modeling approach

__Before running this notebook:__ You will have to download and unzip the bnn samples, chains, and datasets that can be found here (TODO). Because we already have the BNN samples, the model weights are not neccesary.

## Comparison of Full and GMM results for 0.1% Dropout

First, we need to load all of our forward modeling and BNN samples for all three of our BNN models. Our forward modeling is done directly through lenstronomy. To get lenstronomy to load the weights for the forward modeling, we will have to sample once. This should take at most a second or two.

In [ ]:
root_path = os.getcwd()[:-5]

# Grab the config paths for our three BNN models
nn1_config_path = root_path + 'configs/nn1_hr.json'
nn2_config_path = root_path + 'configs/nn2_slr.json'
nn3_config_path = root_path + 'configs/nn3_slr.json'

# Load the config for our three models.
nn1_cfg = model_trainer.load_config(nn1_config_path)
nn2_cfg = model_trainer.load_config(nn2_config_path)
nn3_cfg = model_trainer.load_config(nn3_config_path)

recursive_str_checker(nn1_cfg)
recursive_str_checker(nn2_cfg)
recursive_str_checker(nn3_cfg)

# Samples are already generated so we don't need the model weights
lite_class = True

fow_model_nn1 = forward_modeling.ForwardModel(nn1_cfg,lite_class=lite_class)
fow_model_nn2 = forward_modeling.ForwardModel(nn2_cfg,lite_class=lite_class)
fow_model_nn3 = forward_modeling.ForwardModel(nn3_cfg,lite_class=lite_class)

We need to select the image we'll use for our forward modeling. So long as you've downloaded the datasets or generated them using the baobab configs provided in this git repo, image index 40 will have the correct meaning. This should print out the same image and information 3 times. Note the random noise is fixed by a set seed to ensure that the forward modeling and the BNN see the exact same image.

In [ ]:
# Select the image we want to forward model.
image_index = 40
fow_model_nn1.select_image(image_index)
fow_model_nn2.select_image(image_index)
fow_model_nn3.select_image(image_index)

Now we need to initialize our three forward modeling samplers. They are all pulling the same weights (since the forward model doesn't care which BNN was used).

In [ ]:
# Initialize our sampler for the three models
walker_ratio = 50
n_samps = 1
save_path_chains = os.path.join(root_path,'forward_modeling/%s.h5'%(fow_model_nn1.true_values['img_filename'][:-4]))

fow_model_nn1.initialize_sampler(walker_ratio,save_path_chains)
fow_model_nn1.run_sampler(n_samps)
fow_model_nn2.initialize_sampler(walker_ratio,save_path_chains)
fow_model_nn2.run_sampler(n_samps)
fow_model_nn3.initialize_sampler(walker_ratio,save_path_chains)
fow_model_nn3.run_sampler(n_samps)

Now we specify the path to the pre-run BNN samples. If you have the weights downloaded, feel free to rerun this on a GPU. Even without a GPU it should only take tens of seconds.

In [ ]:
num_samples = 10000
burnin = 2000

sample_save_dir_nn1 = os.path.join(root_path,'forward_modeling/nn1_%s_samps'%(
    fow_model_nn1.true_values['img_filename'][:-4]))
sample_save_dir_nn2 = os.path.join(root_path,'forward_modeling/nn2_%s_samps'%(
    fow_model_nn1.true_values['img_filename'][:-4]))
sample_save_dir_nn3 = os.path.join(root_path,'forward_modeling/nn3_%s_samps'%(
    fow_model_nn1.true_values['img_filename'][:-4]))

Now we can plot the comparison of the forward modeling posterior and the full / GMM posterior.

In [ ]:
# Now let's look at the corner plot of the parameters we care about (the lens parameters)
color_map = ['#d95f02','#7570b3','#000000']
truth_color = '#e7298a'
plot_limits = [(0.03,0.09),(-0.06,0.00),(0.03,0.06),(-0.06,-0.03),(-0.03,0.04),(-0.26,-0.19),(1.6,1.85),
              (-0.04,-0.008)]
save_fig_path = 'figures/fow_model_comp.pdf'


fig = fow_model_nn2.plot_posterior_contours(burnin,num_samples,sample_save_dir=sample_save_dir_nn2,
                                            color_map=color_map,plot_limits=plot_limits,truth_color=truth_color,
                                            save_fig_path=None,show_plot=False,plot_fow_model=False,add_legend=False)
fig = fow_model_nn3.plot_posterior_contours(burnin,num_samples,sample_save_dir=sample_save_dir_nn3,
                                            color_map=color_map[1:],plot_limits=plot_limits,truth_color=truth_color,
                                            save_fig_path=None,fig=fig,show_plot=False,add_legend=False)

handles = [Line2D([0], [0], color=color_map[0], lw=10),
           Line2D([0], [0], color=color_map[1], lw=10),
           Line2D([0], [0], color=color_map[2], lw=10)]
fig.legend(handles,[r'Full BNN 0.1% Dropout',r'GMM BNN 0.1% Dropout','Forward Modeling'],loc=(0.525,0.73),
    fontsize=20,framealpha=1.0)
fig.savefig(save_fig_path)

## Comparison for Diagonal BNN 30% Dropout

We want to generate the same plot as above, but now with the diagaonal BNN model.

In [ ]:
# Now let's look at the corner plot of the parameters we care about (the lens parameters)
color_map = ['#1b9e77','#000000']
truth_color = '#e7298a'
plot_limits = [(0.00,0.09),(-0.05,0.04),(0.01,0.08),(-0.08,-0.01),(-0.07,0.04),(-0.34,-0.16),(1.65,2.1),
              (-0.08,0.01)]
save_fig_path = 'figures/fow_model_comp_diag.pdf'


fig = fow_model_nn1.plot_posterior_contours(burnin,num_samples,sample_save_dir=sample_save_dir_nn1,
                                            color_map=color_map,plot_limits=plot_limits,truth_color=truth_color,
                                            save_fig_path=None,show_plot=False,add_legend=False)

handles = [Line2D([0], [0], color=color_map[0], lw=10),
           Line2D([0], [0], color=color_map[1], lw=10)]
fig.legend(handles,[r'Diagonal BNN 30% Dropout','Forward Modeling'],loc=(0.54,0.73),fontsize=20,framealpha=1.0)
fig.savefig(save_fig_path)